# Topic Modeling on English-language 'Russian Troll Tweets'

Does topic modeling using LDA make sense on short-form data such as tweets?
[Maybe yes](https://www.researchgate.net/post/What_is_a_good_way_to_perform_topic_modeling_on_short_text). Let's try.

In [13]:
import os
# suppress the numerous deprecation warnings
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# text preprocessing
import re
import string
import pandas as pd
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# topic modeling
from gensim.corpora import Dictionary
from gensim.models import LdaModel, Phrases, phrases
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# visualization
import pyLDAvis
from pyLDAvis import gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# displaying the vis right in our notebook
pyLDAvis.enable_notebook()

## Import what we'll work with

In [4]:
data_dir = 'data'
files = os.listdir(data_dir)

In [5]:
# change to loop over all of them
%time df = pd.read_csv(f"{data_dir}/{files[0]}")

CPU times: user 2.18 s, sys: 371 ms, total: 2.55 s
Wall time: 2.84 s


## Inspect the data quickly

In [6]:
df.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer
3,1.674084e+09,GAB1ALDANA,#MyAchillesHeel slippery floors https://t.co/R...,United States,Norwegian,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2160,RETWEET,Hashtager,0,1,HashtagGamer
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer


## Filter for English language tweets

We want to look at "Russians covertly posing as English-speakers", therefore we'll only look at tweets that have `English` as a `language`.

Another option would be to search for `region` and select only those coming from `United States`, but IMO the previous filter would be more interesting.

In [7]:
en_df = df[df.language == 'English']

## Text preprocessing and data cleaning

Lesson learned from yesterday, let's only pull out what we really don't want to happen inside the function!

In [8]:
# instantiating our multi-use tokenizer
tknzr = TweetTokenizer()

# creating the punctuation list we want to exclude
punct = string.punctuation
# adding additional common punctuation chars of the texts
add_punct = ""
punct += add_punct
    
# our extended stopwords list
stpw = stopwords.words('english')
add_stopwords = ['http', 'https']
stpw.extend(add_stopwords)

Finally it's time to introduce RegExp to boost our matching :)

In [9]:
# matches two or more alpha characters
# thus it should exclude things such as 's or numbers, or any single-letters floating around
# however, there will still be a match if we run it on e.g. #usa
# play here: https://regexr.com/
regexp = re.compile(r'[a-z]{2,}')

In [10]:
def preprocess(tweet, tokenizer, regexp, punct, stpw):
    # remove capitalization
    tweet = tweet.lower()
    # tokenize
    tokens = tokenizer.tokenize(tweet)
    # remove punctuation ('t' stands for 'token' - we're looping over all tokens)
    no_punct = (t for t in tokens if not t in punct)
    # remove stopwords
    no_stpw = (t for t in no_punct if not t in stpw)
    # remove other strange character-letter-punctuation combinations
    # NOTE: this will filter out things such as emojis and text-based emoticons
    # TODO: adapt the RegExp above to keep matching those!
    no_weirds = (t for t in no_stpw if re.search(regexp, t))
    # lemmatize remaining tokens
    lem = WordNetLemmatizer()
    lem_tokens = [lem.lemmatize(t) for t in no_weirds]
    return lem_tokens

In [11]:
%time tweet_corpus = [preprocess(tweet, tknzr, regexp, punct, stpw) for tweet in iter(df.content) if type(tweet) == str]

CPU times: user 1min 23s, sys: 1.33 s, total: 1min 24s
Wall time: 1min 28s


# Creating Bigrams and Trigrams

Should work with a collection of texts.

In [25]:
# Build the bigram and trigram models
bigram = Phrases(tweet_corpus, min_count=10, threshold=200) # higher threshold fewer phrases.
trigram = Phrases(bigram[tweet_corpus], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = phrases.Phraser(bigram)
trigram_mod = phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[tweet_corpus[0]]])

/Users/martin/Documents/codingnomads/nlpython/big_projects/troll_tweets/env/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['people', 'toxic', 'think', 'people', 'poisoning']


In [26]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [27]:
# Form Bigrams
%time data_words_bigrams = make_bigrams(tweet_corpus)

CPU times: user 31.3 s, sys: 10.8 s, total: 42.1 s
Wall time: 1min 46s


In [28]:
print(data_words_bigrams[:20])

[['people', 'toxic', 'think', 'people', 'poisoning'], ['#nowplaying', 'shoot', 'man', '@devo', 'https://t.co/9ildvpexkb', 'buy', 'https://t.co/6gkovvcur0'], ['boring', 'person', 'world', 'starterpack', '#pokemongo', 'https://t.co/u8woa1s3j7'], ['#myachillesheel', 'slippery', 'floor', 'https://t.co/r8nqnxnx4l'], ['#myachillesheel', 'boring', 'narcissist', 'nothing', 'wrong', 'narcissism', 'boring', 'dare'], ['opinion', 'hillary', 'really', 'matter', 'non-american', 'https://t.co/tze6denkr0'], ['#myachillesheel', 'lilith', 'frasier'], ['come', 'find', 'u', 'national', 'mall', '#dc', '#pokewalk', 'treat', 'thanks', '@kindsnacks', '@drinkbai', '#pokemon', 'https://t.co/teofzdwm66'], ['#myachillesheel', 'trolling', 'celebrity', 'blocked', 'wil', 'wheathead', 'explain', '#nra', 'caused', "earth's", 'evil'], ['#myachillesheel', 'morbid', 'comedy'], ['#myachillesheel', 'yo', 'momma', 'beyonce', 'costume'], ['#myachillesheel', 'woman', 'heel', 'https://t.co/ycggu01btv'], ['sometimes', 'seeing',

In [29]:
bigrams = []

for word_li in data_words_bigrams:
    for w in word_li:
        if "_" in w:
            bigrams.append(w)

In [30]:
# bigrams[:30]  # old_fashioned

In [31]:
len(set(bigrams))

7669

## Topic modeling with `gensim`

In [35]:
%time dictionary = Dictionary(tweet_corpus)

CPU times: user 9.45 s, sys: 628 ms, total: 10.1 s
Wall time: 10.5 s


In [36]:
%time gen_corpus = [dictionary.doc2bow(tweet) for tweet in tweet_corpus]

CPU times: user 7.91 s, sys: 2.27 s, total: 10.2 s
Wall time: 13.2 s


After timing the different cell execution times with `%time`, it becomes clear that the main time-eater here is building the LDA model. We can't really speed that up without having a good understanding of the internal workings of `gensim`.

In [37]:
warnings.filterwarnings("ignore")
%time ldamodel = LdaModel(corpus=gen_corpus, num_topics=10, id2word=dictionary)

CPU times: user 4min 55s, sys: 15.9 s, total: 5min 10s
Wall time: 5min 25s


In [38]:
ldamodel.show_topics()

[(0,
  '0.015*"#pjnet" + 0.012*"#tcot" + 0.009*"obama" + 0.008*"u" + 0.007*"like" + 0.006*"medium" + 0.006*"help" + 0.006*"people" + 0.006*"say" + 0.005*"report"'),
 (1,
  '0.008*"rt" + 0.008*"killing" + 0.008*"missing" + 0.008*"cruz" + 0.007*"texas" + 0.007*"terror" + 0.006*"return" + 0.006*"die" + 0.006*"driver" + 0.006*"district"'),
 (2,
  '0.042*"#news" + 0.034*"police" + 0.018*"man" + 0.012*"shooting" + 0.009*"school" + 0.009*"officer" + 0.007*"suspect" + 0.007*"shot" + 0.007*"arrested" + 0.007*"topeka"'),
 (3,
  '0.007*"supreme" + 0.007*"sentenced" + 0.007*"congress" + 0.006*"mother" + 0.005*"air" + 0.005*"train" + 0.005*"#job" + 0.005*"oklahoma" + 0.005*"protect" + 0.004*"east"'),
 (4,
  '0.054*"kansa" + 0.035*"#news" + 0.015*"american" + 0.010*"county" + 0.010*"city" + 0.010*"death" + 0.008*"new" + 0.008*"tax" + 0.006*"charged" + 0.006*"missouri"'),
 (5,
  '0.019*"gun" + 0.008*"#isis" + 0.006*"control" + 0.006*"targeted" + 0.006*"freedom" + 0.006*"islam" + 0.006*"#opiceisis" + 

## Visualize the resulting topics

Here we are using `pyLDAvis` - the `import` needs to be explicit to work, so it might appear that this is part of the `gensim` package, but instead this is from `pyLDAvis`.

In [39]:
%time gensim.prepare(ldamodel, gen_corpus, dictionary)

CPU times: user 1min 16s, sys: 5.57 s, total: 1min 21s
Wall time: 2min 37s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.206197  0.207676       1        1  17.336222
6     -0.228125  0.042516       2        1  15.270926
7     -0.069394  0.139111       3        1  10.850449
2     -0.066423 -0.194469       4        1   9.686235
8     -0.094625 -0.182105       5        1   9.507814
4     -0.017407 -0.181927       6        1   9.014377
1      0.168699  0.044342       7        1   8.533293
5      0.160709  0.040920       8        1   6.687271
3      0.178764  0.041472       9        1   6.609138
9      0.174001  0.042463      10        1   6.504275, topic_info=      Category          Freq               Term         Total  loglift  \
term                                                                     
9495   Default  49217.000000              #news  49217.000000  30.0000   
8716   Default  14427.000000              kansa  14427.000000  29.0000   
86     Default  21692.000000              trump  21692.000000  28.0000   
585    Default   9653.000000             police   9653.000000  27.0000   
1049   Default   8113.000000            clinton   8113.000000  26.0000   
9      Default   9485.000000                man   9485.000000  25.0000   
26     Default   7389.000000            hillary   7389.000000  24.0000   
10354  Default   7629.000000             #pjnet   7629.000000  23.0000   
7917   Default   8195.000000          #politics   8195.000000  22.0000   
9918   Default   6141.000000              #tcot   6141.000000  21.0000   
93     Default   8226.000000                new   8226.000000  20.0000   
3360   Default   3654.000000                gun   3654.000000  19.0000   
581    Default   3911.000000           american   3911.000000  18.0000   
390    Default   7269.000000              obama   7269.000000  17.0000   
3568   Default   3678.000000                 mt   3678.000000  16.0000   
583    Default   4478.000000               city   4478.000000  15.0000   
3247   Default   4522.000000           shooting   4522.000000  14.0000   
0      Default   6733.000000             people   6733.000000  13.0000   
839    Default   5754.000000              state   5754.000000  12.0000   
9002   Default   3170.000000              crash   3170.000000  11.0000   
1153   Default   3921.000000              house   3921.000000  10.0000   
914    Default   3962.000000             killed   3962.000000   9.0000   
61011  Default   3792.000000            wichita   3792.000000   8.0000   
979    Default   3041.000000               bill   3041.000000   7.0000   
8199   Default   2769.000000             county   2769.000000   6.0000   
1511   Default   3529.000000            america   3529.000000   5.0000   
114    Default   3639.000000              death   3639.000000   4.0000   
161    Default   4820.000000                 rt   4820.000000   3.0000   
223    Default   2626.000000             attack   2626.000000   2.0000   
980    Default   2773.000000               call   2773.000000   1.0000   
...        ...           ...                ...           ...      ...   
2798   Topic10   1796.910034            weather   1797.604370   2.7323   
8378   Topic10   1510.374634            accused   1511.068970   2.7323   
666    Topic10   1098.900391               rain   1099.594727   2.7321   
6197   Topic10   1039.090820             recall   1039.785156   2.7320   
1310   Topic10    970.595886              local    971.290222   2.7320   
1307   Topic10    931.381531             center    932.075867   2.7320   
1330   Topic10    820.617615            outside    821.311890   2.7319   
1505   Topic10    806.201782               hear    806.896362   2.7318   
1335   Topic10    766.111206  #blacklivesmatter    766.805420   2.7318   
2616   Topic10    735.373474            station    736.067810   2.7318   
1431   Topic10    679.963379            message    680.657654   2.7317   
3096   Topic10    672.685791             safety    67

## Ideas / Todos

* run this analysis on the full corpus (one file takes approximately 8-9 minutes to process)
* cherry-pick some words to include into the stopword list
* ~~adapt the tokenizer to avoid splitting `#` symbols from twitter hashtags~~
* reduce run-time

## Notes

`u` is probably still in the results because we are running the **lemmatization** (`WordNetLemmatizer`) after filtering - therefore something such as `ur` (which would pass our RegExp criteria, because it's 2 characters) might get stemmed into `u` and remain in the results.